# TableauSimplex

In [ ]:
#using Pkg; Pkg.add("https://github.com/krislock/TableauSimplex.jl.git")

In [ ]:
using TableauSimplex

# Nodes/Vertices
V = [1, 2, 3, 4, 5, 6]

# Arcs/Edges
E = [(1,2), (1,3), (1,5), (2,3), (2,4), (3,4), (3,5), (4,6), (5,6)]

# Demands
b = [-1, -3, 0, 0, 0, 4]

# Costs
c = [2, 3, 3, 2, 4, 1, 2, 3, 1]

m, n = length(V), length(E)

# Phase I

In [ ]:
# Add artificial node m+1 with directed edges 
# from vertices with negative demands and to 
# vertices with positive demands

E0 = copy(E)
for i in V
    if b[i] < 0
        push!(E0, (i, m+1))
    else
        push!(E0, (m+1, i))
    end
end
E0

In [ ]:
# Create the incidence matrix A0

A0 = zeros(eltype(b), m+1, length(E0))
for (k, e) in enumerate(E0)
    i, j = e
    A0[i,k] = -1
    A0[j,k] = 1
end
b0 = [b; 0]
[A0 b0]

In [ ]:
table = Rational{Int}[ zeros(1, n) ones(1, size(A0,2)-n) 0; A0[1:end-1,:] b0[1:end-1] ]
basis = collect(n+1:n+m)
varnames = ["x$i$j" for (i,j) in E]
artificials = ["a$i" for i in V]
T1 = Tableau(table, basis, [varnames; artificials])

In [ ]:
clean!(T1)

In [ ]:
simplex!(T1, verbose=true);

In [ ]:
println("Basic feasible solution:")
optsoln = [E0[T1.basis] rhs(T1)]

# Phase II

In [ ]:
# Drop all artificial variables and use original edge costs
inds = [i for i=1:m if T1.basis[i] <= n]
table = [
    c' 0
    T1.table[inds.+1, 1:n] T1.table[inds.+1, end]
]
basis = T1.basis[inds]
T2 = Tableau(table, basis, varnames)

In [ ]:
clean!(T2)

In [ ]:
simplex!(T2, verbose=true);

In [ ]:
println("Optimal solution:")
optsoln = [E[T2.basis] rhs(T2)]